In [1]:
import numpy as np
import itertools

# Tests

## average value of function

We compute the average value of $f(x)=x^{2}$ on the interval 0 to 4.  This is given by $\int_{0}^{n} dx\, \frac{x^{2}}{n}=\frac{n^{2}}{3}$.

In [2]:
n=5

In [3]:
# f(x)=x**2
def x_squared(x):
    return x**2

In [4]:
# sample 10**5 points uniformly in the range 0 to 4 and append to my vals_array
vals_array = np.empty(0)
for _ in range(10**5):
    num = n*np.random.rand(1)
    vals_array = np.append(vals_array, x_squared(num))

In [5]:
# average value of vals_array is approximately 16/3 = 5.333; uncertainty is standard deviation
# print central_val - 1 std, central_val, central_val + 1 std
mean_val = vals_array.mean()
std_val = vals_array.std()/np.sqrt(len(vals_array))
print(mean_val-std_val, mean_val, mean_val+std_val)

8.31390617891351 8.337513410330516 8.361120641747522


In [6]:
# exact answer
(n**2)/3

8.333333333333334

## $\mathbb{Z}_{n}$ approximation of $U(1)$

We compute the average value of the group element $e^{i\phi}$ in the interval where it is nearer the identity element than to any other element of $\mathbb{Z}_{n}$.  This is given by $\int_{-\pi/n}^{\pi/n}d\phi\, \frac{e^{i\phi}}{2\pi/n} = \frac{n}{\pi}\sin{\frac{\pi}{n}}$

In [7]:
n=2

In [8]:
def xi_r(phi):
    return np.exp(1j*phi)

In [9]:
# sample 10**5 points uniformly in the range 0 to 4 and append to my vals_array
vals_array = np.empty(0)
for _ in range(10**5):
    phi = (2*np.pi/n)*np.random.rand(1)-(np.pi/n)
    vals_array = np.append(vals_array, xi_r(phi))

In [10]:
vals_array.mean()

np.complex128(0.6356249219948629-0.0031246296743675j)

In [11]:
(n/np.pi)*np.sin(np.pi/n)

np.float64(0.6366197723675814)

# Generate S(1080)

In [2]:
omega = np.exp(2*np.pi*1j/3)
mu_p = (1/2)*(-1+np.sqrt(5))
mu_m = (1/2)*(-1-np.sqrt(5))
gA = np.array([[0,1,0],[0,0,1],[1,0,0]])
gF = np.array([[1,0,0],[0,-1,0],[0,0,-1]])
gH = (1/2)*np.array([[-1, mu_m, mu_p], [mu_m, mu_p, -1], [mu_p, -1, mu_m]])
gQ = np.array([[-1,0,0],[0,0,-omega],[0,-omega**2,0]])

In [51]:
S_1080_start = np.array([np.identity(3), *[np.linalg.matrix_power(gA, i) for i in range(1, 3)], gF, gH, gQ,\
                    gF@gQ, gA@gH, gA@gF, gF@gH, gH@gQ, *[np.linalg.matrix_power(gA@gQ, i) for i in range(1, 7)]])
S_1080_start = np.round(S_1080_start, 6)
S_1080_start = np.unique(S_1080_start ,axis=0)

In [52]:
S_1080 = S_1080_start
S_1080_temp = S_1080_start

In [53]:
S_1080.shape

(16, 3, 3)

In [54]:
pairs=itertools.product(S_1080, repeat=2)

for i, item in enumerate(pairs):
    elem = np.round(item[0]@item[1], 6)
    if elem in S_1080_start:
        for j, item in enumerate(S_1080_temp):
            print(i, j)
            if np.linalg.svd(item - elem).S.max() < 10**-4:
                S_1080_start = np.append(S_1080_start, np.array([elem]), axis=0)
                break

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
11 13
11 14
11 15
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
12 12
12 13
12 14
12 15
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
13 13
13 14
13 15
14 0
14 1
15 0
16 0
16 1
16 2
1

In [55]:
S_1080.shape

(16, 3, 3)

In [56]:
S_1080_start.shape

(116, 3, 3)

In [57]:
S_1080_start=np.unique(S_1080_start, axis=0)

In [58]:
S_1080_start.shape

(44, 3, 3)

In [ ]:
should_break = False
for i in range(100):
    if (i<10) and (not should_break):
        print(i)
        

In [2]:
my_array = np.array([*range(1, 6)])
other_array = my_array

for i, item in enumerate(my_array):
    for j, jtem in enumerate(other_array):
        print(i, j, len(other_array))
        other_array = np.append(other_array, jtem)

0 0 5
0 1 6
0 2 7
0 3 8
0 4 9
1 0 10
1 1 11
1 2 12
1 3 13
1 4 14
1 5 15
1 6 16
1 7 17
1 8 18
1 9 19
2 0 20
2 1 21
2 2 22
2 3 23
2 4 24
2 5 25
2 6 26
2 7 27
2 8 28
2 9 29
2 10 30
2 11 31
2 12 32
2 13 33
2 14 34
2 15 35
2 16 36
2 17 37
2 18 38
2 19 39
3 0 40
3 1 41
3 2 42
3 3 43
3 4 44
3 5 45
3 6 46
3 7 47
3 8 48
3 9 49
3 10 50
3 11 51
3 12 52
3 13 53
3 14 54
3 15 55
3 16 56
3 17 57
3 18 58
3 19 59
3 20 60
3 21 61
3 22 62
3 23 63
3 24 64
3 25 65
3 26 66
3 27 67
3 28 68
3 29 69
3 30 70
3 31 71
3 32 72
3 33 73
3 34 74
3 35 75
3 36 76
3 37 77
3 38 78
3 39 79
4 0 80
4 1 81
4 2 82
4 3 83
4 4 84
4 5 85
4 6 86
4 7 87
4 8 88
4 9 89
4 10 90
4 11 91
4 12 92
4 13 93
4 14 94
4 15 95
4 16 96
4 17 97
4 18 98
4 19 99
4 20 100
4 21 101
4 22 102
4 23 103
4 24 104
4 25 105
4 26 106
4 27 107
4 28 108
4 29 109
4 30 110
4 31 111
4 32 112
4 33 113
4 34 114
4 35 115
4 36 116
4 37 117
4 38 118
4 39 119
4 40 120
4 41 121
4 42 122
4 43 123
4 44 124
4 45 125
4 46 126
4 47 127
4 48 128
4 49 129
4 50 130
4 51 131
4 

In [426]:
other_array

array([1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2,
       3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4,
       5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1,
       2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3,
       4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5,
       1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2,
       3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4,
       5, 1, 2, 3, 4, 5])